In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Dense, dot, Reshape , Input
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import pandas as pd 

In [4]:
data = pd.read_csv("corpus_100k", sep='։\n', names = ['sentence','-'])

/var/folders/9f/r8h0fl6158l8g2_8l65x2zv00000gn/T/ipykernel_5127/2051259223.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv("corpus_100k", sep='։\n', names = ['sentence','-'])


In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['sentence'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

In [11]:
# Generate skip-gram training samples with negative sampling
def generate_skipgrams(sentences, vocab_size, window_size=4, negative_samples=1.0, seed=42):
    couples, labels = skipgrams(
        sentences,
        vocabulary_size=vocab_size,
        window_size=window_size,
        negative_samples=negative_samples,
        shuffle=True,
        seed=seed
    )
    word_target, word_context = zip(*couples)
    word_target = np.array(word_target, dtype="int32").reshape(-1, 1)
    word_context = np.array(word_context, dtype="int32").reshape(-1, 1)
    labels = np.array(labels, dtype="int32")
    return word_target, word_context, labels


In [ ]:
window_size = 4
negative_samples = 5  

word_target, word_context, labels = generate_skipgrams(
    tokenizer.texts_to_sequences(data['sentence']),
    vocab_size,
    window_size=window_size,
    negative_samples=negative_samples
)

#  the word2vec 
embedding_size = 100 
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=1))
model.add(Reshape((embedding_size,)))
model.add(Dense(embedding_size, activation='sigmoid'))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss=negative_sampling_loss, optimizer='adam')

model.fit([word_target, word_context], labels, epochs=10, batch_size=64)

embedding_layer_weights = model.layers[0].get_weights()[0]


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# t-SNE to reduce the dimensionality of the embeddings to 2D
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embedding_layer_weights)

# Visualize 
plt.figure(figsize=(10, 10))
for i in range(vocabulary_size):
    plt.scatter(embeddings_2d[i, 0], embeddings_2d[i, 1])
    plt.annotate(word_index_mapping[i], xy=(embeddings_2d[i, 0], embeddings_2d[i, 1]), 
                 xytext=(5, 2), textcoords='offset points', ha='right', va='bottom')

plt.show()